<a href="https://colab.research.google.com/github/sudamah/sentence-similarity/blob/main/resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Resume Parser** 

*   **Will Use Transformer and Spacy to Train and Predict the Model**
*   **NER**





In [ ]:
!pip install spacy_transformers
!pip install -U spacy
!pip install PyMuPDF

In [2]:
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
!nvidia-smi

Mon Apr  3 16:49:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    26W /  70W |    363MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
cv_data = json.load(open('/content/resume_parser/data/training/train_data.json', 'r'))

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config /content/resume_parser/data/training/base_config.cfg /content/resume_parser/data/training/config.cfg

2023-04-03 16:49:30.410850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-04-03 16:49:30.410955: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-04-03 16:49:30.410973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/resume_parser/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy 

In [12]:
def get_spacy_doc(data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue

            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue
            
            # if span is None:
            #     err_data = str([start, end]) + "   " + str(text) + '\n'
            #     file.write(err_data)

            if span is not None:
                ents.append(span)
        
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db







In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.30)

In [14]:
len(train), len(test)

(140, 60)

In [ ]:
# file = open('error.txt', 'r')

In [15]:
db = get_spacy_doc(train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(test)
db.to_disk('test_data.spacy')

100%|██████████| 60/60 [00:00<00:00, 63.54it/s]


In [17]:
len(db.tokens)

60

In [19]:
!python -m spacy train /content/resume_parser/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-04-03 17:06:31.173265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-04-03 17:06:31.173401: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-04-03 17:06:31.173423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-03 17:06:37,092] [INFO] Set

### **Model Test**

In [20]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
!zip -r /content/output.zip /content/output

In [29]:
from google.colab import files
files.download("/content/output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import sys, fitz

In [22]:
fname = '/content/resume_parser/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [23]:
text = " "
for page in doc:
    text = text + str(page.get_text())

In [24]:
text

' Alice Clark \nAI / Machine Learning \n \nDelhi, India Email me on Indeed \n• \n20+ years of experience in data handling, design, and development \n• \nData Warehouse: Data analysis, star/snow flake scema data modelling and design specific to \ndata warehousing and business intelligence \n• \nDatabase: Experience in database designing, scalability, back-up and recovery, writing and \noptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, \nStream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake \nanalytics(U-SQL) \nWilling to relocate anywhere \n \nWORK EXPERIENCE \nSoftware Engineer \nMicrosoft – Bangalore, Karnataka \nJanuary 2000 to Present \n1. Microsoft Rewards Live dashboards: \nDescription: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping \nonline. Microsoft Rewards members can earn points when

In [25]:
text = " ".join(text.split())

In [27]:
doc = nlp(text)
for ent in  doc.ents:
    print(ent.label_,  '----------', ent.text)

Name ---------- Alice Clark
Degree ---------- AI / Machine Learning
Location ---------- Delhi
Years of Experience ---------- 20+ years
Companies worked at ---------- Microsoft
Designation ---------- Software Engineer
Companies worked at ---------- Microsoft
Location ---------- Bangalore
Companies worked at ---------- Microsoft
Companies worked at ---------- Microsoft
Companies worked at ---------- Microsoft
Companies worked at ---------- Microsoft
Companies worked at ---------- Microsoft
Companies worked at ---------- Microsoft
College Name ---------- Indian Institute of Technology
Location ---------- Mumbai
Skills ---------- Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal skills with ability to interact with individuals at all the levels • Quick learner and maintains cordial relationship with project manager and team members and g